<a href="https://colab.research.google.com/github/savindumahasen/Skin-Disease-Classification-Project/blob/main/Skin_disease_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## install the necessary libraries
!pip install numpy -q
!pip install pandas -q
!pip install matplotlib -q
!pip install tensorflow -q
!pip install opendatasets -q

In [3]:
## Import the necessary libraries
import opendatasets as od
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

In [4]:
## download the dataset
od.download("https://www.kaggle.com/datasets/pritpal2873/multiple-skin-disease-detection-and-classification")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: savinduruhunuhewa
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/pritpal2873/multiple-skin-disease-detection-and-classification


100%|██████████| 785M/785M [00:08<00:00, 95.9MB/s]


In [5]:
## Define the batch size and image size
BATCH_SIZE=32
IMAGE_SIZE=(128,128)

In [51]:
## Load the dataset and split into train and test set

skin_cancer_dataset = "/content/multiple-skin-disease-detection-and-classification/Skin-Cancer-Dataset/"

dataset_train=tf.keras.utils.image_dataset_from_directory(skin_cancer_dataset ,
                                                    batch_size=BATCH_SIZE,
                                                    image_size=IMAGE_SIZE,
                                                    subset='training',
                                                    validation_split=0.2,
                                                    seed=42)

dataset_test=tf.keras.utils.image_dataset_from_directory(skin_cancer_dataset,
                                                         subset='validation',
                                                         batch_size=BATCH_SIZE,
                                                         image_size=IMAGE_SIZE,
                                                         validation_split=0.2,
                                                         seed=42)





Found 4107 files belonging to 9 classes.
Using 3286 files for training.
Found 4107 files belonging to 9 classes.
Using 821 files for validation.


AttributeError: '_PrefetchDataset' object has no attribute 'shape'

In [ ]:
/content/multiple-skin-disease-detection-and-classification/Skin-Cancer-Dataset/Acitinic-Keratosis